In [1]:
#Dataset source: https://www.kaggle.com/datasets/subhajournal/free-spoken-digit-database/code
#loading data: https://github.com/SeanNaren/deepspeech.pytorch/blob/master/deepspeech_pytorch/loader/data_loader.py
import os
import torch
import torchaudio
import IPython
import librosa
#from hmmlearn import hmm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, Sampler, DistributedSampler, DataLoader


In [11]:
path = 'archive/'

def extract_mfcc(file_path, num_mfcc=39, n_fft=512, hop_length=80):
    signal, rate = librosa.load(file_path, sr=16000)
    mfcc_features = librosa.feature.mfcc(y=signal, sr=rate, n_mfcc=num_mfcc, n_fft=n_fft,  hop_length= hop_length)
    return mfcc_features.T

#extracts Mel-frequency cepstral coefficients(MFCC)
def preprocess_digit_data(digit_folder, num_mfcc=39):
    digit_mfccs = []
    for filename in os.listdir(digit_folder):
        if filename.endswith(".wav"):
            file_path = os.path.join(digit_folder, filename)
            mfcc_features = extract_mfcc(file_path, num_mfcc)
            digit_mfccs.append(mfcc_features)
    return digit_mfccs

class_dir = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
#print(class_dir)

x = []
y = []
for p in class_dir:
    x.append(preprocess_digit_data(path + p))
    length = len([name for name in os.listdir(path + p)])
    for k in range(0, length):
        y.append(p)

#data = preprocess_digit_data(classes[0])

In [12]:
print(x[1])
print(y[1])

[array([[-8.1796960e+02,  3.3465216e+00,  3.0630622e+00, ...,
         1.4070637e+00,  1.3394629e+00,  1.2375021e+00],
       [-8.1436603e+02,  7.6863289e+00,  5.6377573e+00, ...,
         1.5224984e+00,  1.0754176e+00,  9.9393553e-01],
       [-8.1436414e+02,  7.6884556e+00,  5.6728401e+00, ...,
         1.1112200e+00,  2.5750321e-01, -3.4042299e-02],
       ...,
       [-8.1258093e+02,  2.0372183e+00, -5.2916737e+00, ...,
        -3.8532331e+00, -1.6710714e+00,  7.1371436e-01],
       [-8.1296735e+02,  2.1582901e+00, -4.7587833e+00, ...,
        -4.3665428e+00, -4.8213214e-01,  1.6909282e+00],
       [-8.1526025e+02,  3.5552657e-01, -5.3419847e+00, ...,
        -3.1111808e+00,  1.5858901e-01,  2.3144941e+00]], dtype=float32), array([[-945.60535,    0.     ,    0.     , ...,    0.     ,    0.     ,
           0.     ],
       [-945.60535,    0.     ,    0.     , ...,    0.     ,    0.     ,
           0.     ],
       [-945.60535,    0.     ,    0.     , ...,    0.     ,    0.     ,
 

In [19]:
class custom_dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.len = x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return self.len
    
data = custom_dataset(x, y)

1700
